In [1]:
# 代码参考来源 https://huggingface.co/blog/fine-tune-wav2vec2-english
from datasets import load_dataset, load_metric

timit = load_dataset("timit_asr")
print(timit)

In [2]:
timit = timit.remove_columns(["phonetic_detail", "word_detail", "dialect_region", "id", "sentence_type", "speaker_id"])

In [27]:
timit['test']['text']

['She had your dark suit in greasy wash water all year.',
 "Don't ask me to carry an oily rag like that.",
 'Bricks are an alternative.',
 'Fat showed in loose rolls beneath the shirt.',
 'It suffers from a lack of unity of purpose and respect for heroic leadership.',
 'She slipped and sprained her ankle on the steep slope.',
 'Aluminum silverware can often be flimsy.',
 'Young children should avoid exposure to contagious diseases.',
 'Weatherproof galoshes are very useful in Seattle.',
 'Pam gives driving lessons on Thursdays.',
 'She had your dark suit in greasy wash water all year.',
 "Don't ask me to carry an oily rag like that.",
 'His captain was thin and haggard and his beautiful boots were worn and shabby.',
 'The reasons for this dive seemed foolish now.',
 'Production may fall far below expectations.',
 'Pizzerias are convenient for a quick lunch.',
 'Put the butcher block table in the garage.',
 'Drop five forms in the box before you go out.',
 'Her wardrobe consists of only

In [28]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

show_random_elements(timit["train"].remove_columns(["file"]))

,id,text
0,389,Challenge each general's intelligence.
1,2432,Or how do we enrich our already fine marriage?
2,4092,Prompted by a guilty urge
3,2237,A leather handbag would be a suitable gift.
4,2367,Al received a joint appointment in the biology and the engineering departments.
5,4346,My ideal morning begins with hot coffee.
6,1202,One can only speak of what is in front of him
7,3761,Don't ask me to carry an oily rag like that.
8,1135,The mango and the papaya are in a bowl.
9,2896,The jaw operates by using antagonistic muscles.


In [29]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower()
    return batch

timit = timit.map(remove_special_characters)

Loading cached processed dataset at C:\Users\lyj157175\.cache\huggingface\datasets\csv\default-ea8d1ea76546f5b3\0.0.0\2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0\cache-24ded232ae491bfc.arrow
Loading cached processed dataset at C:\Users\lyj157175\.cache\huggingface\datasets\csv\default-ea8d1ea76546f5b3\0.0.0\2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0\cache-7a409ca53f52b94d.arrow


In [30]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

vocabs = timit.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=timit.column_names["train"])

In [31]:
len(vocabs['train'][0]['vocab'])

28

In [32]:
vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))

vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'y': 0,
 'p': 1,
 'b': 2,
 'e': 3,
 ' ': 4,
 's': 5,
 'w': 6,
 'c': 7,
 'v': 8,
 'u': 9,
 'i': 10,
 'g': 11,
 'r': 12,
 'k': 13,
 'j': 14,
 'f': 15,
 't': 16,
 'z': 17,
 'o': 18,
 'q': 19,
 'm': 20,
 'x': 21,
 "'": 22,
 'd': 23,
 'l': 24,
 'a': 25,
 'n': 26,
 'h': 27}

In [33]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [34]:
vocab_dict

{'y': 0,
 'p': 1,
 'b': 2,
 'e': 3,
 's': 5,
 'w': 6,
 'c': 7,
 'v': 8,
 'u': 9,
 'i': 10,
 'g': 11,
 'r': 12,
 'k': 13,
 'j': 14,
 'f': 15,
 't': 16,
 'z': 17,
 'o': 18,
 'q': 19,
 'm': 20,
 'x': 21,
 "'": 22,
 'd': 23,
 'l': 24,
 'a': 25,
 'n': 26,
 'h': 27,
 '|': 4}

In [35]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
print(len(vocab_dict))

30


In [36]:
vocab_dict

{'y': 0,
 'p': 1,
 'b': 2,
 'e': 3,
 's': 5,
 'w': 6,
 'c': 7,
 'v': 8,
 'u': 9,
 'i': 10,
 'g': 11,
 'r': 12,
 'k': 13,
 'j': 14,
 'f': 15,
 't': 16,
 'z': 17,
 'o': 18,
 'q': 19,
 'm': 20,
 'x': 21,
 "'": 22,
 'd': 23,
 'l': 24,
 'a': 25,
 'n': 26,
 'h': 27,
 '|': 4,
 '[UNK]': 28,
 '[PAD]': 29}

In [37]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [38]:
from transformers import Wav2Vec2CTCTokenizer
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2Processor


tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", 
                                 unk_token="[UNK]", 
                                 pad_token="[PAD]", 
                                 word_delimiter_token="|")
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, 
                                             sampling_rate=16000, 
                                             padding_value=0.0, 
                                             do_normalize=True, 
                                             return_attention_mask=False)
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, 
                              tokenizer=tokenizer)

In [39]:
import soundfile as sf
# import librosa


def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = sf.read(batch["file"])
    batch["speech"] = speech_array
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["text"]
    return batch

timit = timit.map(speech_file_to_array_fn, 
                  remove_columns=timit.column_names["train"], 
                  num_proc=4)

Loading cached processed dataset at C:\Users\lyj157175\.cache\huggingface\datasets\csv\default-ea8d1ea76546f5b3\0.0.0\2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0\cache-9699ec7bece7a1d5.arrow
Loading cached processed dataset at C:\Users\lyj157175\.cache\huggingface\datasets\csv\default-ea8d1ea76546f5b3\0.0.0\2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0\cache-fcb6127b737a16b5.arrow


In [40]:
timit

DatasetDict({
    train: Dataset({
        features: ['sampling_rate', 'speech', 'target_text'],
        num_rows: 4620
    })
    test: Dataset({
        features: ['sampling_rate', 'speech', 'target_text'],
        num_rows: 1680
    })
})

In [19]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(timit["train"]))
ipd.Audio(data=np.asarray(timit["train"][rand_int]["speech"]), autoplay=True, rate=16000)

In [20]:
rand_int = random.randint(0, len(timit["train"]))

print("Target text:", timit["train"][rand_int]["target_text"])
print("Input array shape:", np.asarray(timit["train"][rand_int]["speech"]).shape)
print("Sampling rate:", timit["train"][rand_int]["sampling_rate"])

Target text: the girl nodded understandingly
Input array shape: (31847,)
Sampling rate: 16000


In [41]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values

    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

timit_prepared = timit.map(prepare_dataset, remove_columns=timit.column_names["train"], batch_size=8, num_proc=4, batched=True)

D:\sorfware\anaconda\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


# Training & Evaluation

In [36]:
import torch
 
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [39]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [52]:
wer_metric = load_metric("wer")

In [53]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)
    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id
    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [47]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base", 
    gradient_checkpointing=True, 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
)

In [52]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  # output_dir="/content/gdrive/MyDrive/wav2vec2-base-timit-demo",
  output_dir="./wav2vec2-base-timit",
  group_by_length=True,
  per_device_train_batch_size=32,
  evaluation_strategy="steps",
  num_train_epochs=32,
  fp16=True,
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,
)

In [55]:
from transformers import Trainer

trainer = Trainer(
    model=new_model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=timit_prepared["train"],
    eval_dataset=timit_prepared["test"],
    tokenizer=processor.feature_extractor,
)

In [56]:
trainer.train()

Step,Training Loss,Validation Loss


KeyboardInterrupt: 